In [48]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import re
import functools
from tqdm.notebook import tqdm
import subprocess
import glob
from IPython import display as ipythondisplay
from torch.utils.data import Dataset, DataLoader
from torch.distributions import Categorical

In [3]:
def make_datapath_list(rootpath):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    

    path_list = []
    
    for c in tqdm(alphabet):
        target_path = os.path.join(rootpath + c + '/*.abc')
        for path in glob.glob(target_path):
            path_list.append(path)

    return path_list

In [4]:
def extract_song_data(text):
    # extract song from abc notation
    # remove header
    text = re.sub(r'%%.*', '', text)
    text = text.strip()
    text = re.sub(r'\n{2,}',r'\n',text)
    return text

In [5]:
def save_song_to_abc(song, filename):
    save_name = "/result/{}.abc".format(filename)
    with open(save_name, 'w') as f:
        f.write(song)
    
    return filename

def abc2wav(abc_file):
    suf = abc_file.rsrip('.abc')
    cmd = "abc2midi {} -o {}".format(abc_file, suf + ".mid")
    os.system(cmd)
    cmd = "timidity {}.mid -Ow {}.wav".format(suf, suf)

    return os.system()

def play_wav(wav_file):
    return ipythondisplay.Audio(wav_file)

def play_song(song):
    basename = save_song_to_abc(song)
    ret = abc2wav(basename + ".abc")
    if ret == 0:
        return play_wav(basename + ".wav")
    
    return None

In [6]:
cwd = os.getcwd()
rootpath = cwd + '/notation/'
notation_list = make_datapath_list(rootpath)

  0%|          | 0/26 [00:00<?, ?it/s]

In [7]:
songs = []

for notation in tqdm(notation_list):
    with open(notation, 'r') as f:
        text = f.read()
        song = extract_song_data(text)
        songs.append(song)

  0%|          | 0/1744 [00:00<?, ?it/s]

In [8]:
example_song = songs[119]
print("Example song: ")
print(example_song)

Example song: 
X: 0
T: Austria
Z: Franz Joseph Hayden, 1797
Z: Public  domain
L: 1/4
M: 4/4
V: P1 name="Unnamed-000"
V: P2 name="Unnamed-001"
V: P3 name="Tempo Track"
K: Eb
[V: P1]  [B,3/E3/] [B,/F/] [EG] [DF] | [FA] [EG] [D/F/]D/ [B,E] | [C/c/]D/ [EB] [FA] [EG] | [CF] [E/G/]E/ [D2B2] | [B,3/E3/] [B,/F/] [EG] [DF] | [FA] [EG] [D/F/]D/ [B,E] | [C/c/]D/ [EB] [FA] [EG] | [CF] [E/G/]E/ [D2B2] | [B,F] [B,G] [B,/F/]D/ B, | [C/A/]D/ [EG] [D/F/][B,/D/] B, | [B,B] [C/A/]D/ [E3/G3/] [E/G/] | [E3/=A3/] [E/A/] [D2B2] | [E3/e3/] [E/d/] [E/d/]c/ [EB] | [E3/c3/] [E/B/] [D/B/]A/ [EG] | [DF] [D/G/]A/ [E/B/]c/ [C/A/]F/ | [B,E] [D/G/]F/ E2- | E2z2|]
[V: P2]  [E,3/G,3/] [E,/A,/] [E,B,] [B,,B,] | [D,B,] [E,B,] [B,,A,] [E,G,] | A, [G,B,] [D,B,] [E,B,] | [A,,A,] [=A,,C] [B,,2F,2] | [E,3/G,3/] [E,/A,/] [E,B,] [B,,B,] | [D,B,] [E,B,] [B,,A,] [E,G,] | A, [G,B,] [D,B,] [E,B,] | [A,,A,] [=A,,C] [B,,2F,2] | [B,,D,] [B,,E,] [B,,/D,/]F,/ [B,,/A,/]G,/ | [B,,F,] [B,,/E,/]G,/ [B,,/B,/]F,/ [B,,/D,/]A,,/ | [G,,G,] [A,,/F

In [9]:
# define a class to read data
# this class is used to read data from a list of songs and return a data
# the input is a list of songs, sequence length, batch size
# the output is a batch of data

class SongDataset(Dataset):
    def __init__(self, songs, seq_len):
        self.songs = songs
        self.seq_len = seq_len
        self.vocab = set()
        self.vocab_to_int = {}
        self.int_to_vocab = {}
        self.vocab_size = 0
        self.song_index = 0
        self.song = self.songs[self.song_index]
        self.song_len = len(self.song)
        self.song_index += 1
        self.create_vocab()
        self.create_int()
        self.n_batch = int(self.song_len / self.seq_len)
        
    def create_vocab(self):
        for song in self.songs:
            self.vocab = self.vocab.union(set(song))
        self.vocab = sorted(self.vocab)
        self.vocab_size = len(self.vocab)
        
    def create_int(self):
        self.vocab_to_int = {c: i for i, c in enumerate(self.vocab)}
        self.int_to_vocab = dict(enumerate(self.vocab))
        
    def __len__(self):
        return self.n_batch
    
    def __getitem__(self, index):
        
        x_str = self.song[index : index + self.seq_len]
        y_str = self.song[index+1 : index + self.seq_len + 1]

        x = torch.tensor([self.vocab_to_int[c] for c in x_str])
        y = torch.tensor([self.vocab_to_int[c] for c in y_str])

        return x, y

In [10]:
# define a class to build a RNN model
# the input is the number of hidden units, the number of layers, the size of the vocabulary, the batch size, the sequence length

class RNN(nn.Module):
    def __init__(self, n_hidden, n_layers, embedding_dim, vocab_size, batch_size, seq_len):
        super(RNN, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.embed = nn.Embedding(self.vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.n_hidden, self.n_layers, batch_first=True)
        self.fc = nn.Linear(self.n_hidden, self.vocab_size)
        
    def forward(self, x, prev_state):
        x = self.embed(x)
        r_out, hidden = self.lstm(x, prev_state)
        
        out = self.fc(r_out)
        
        return out, hidden
    
    def init_hidden(self):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, self.batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, self.batch_size, self.n_hidden).zero_())
        
        return hidden

In [11]:
# define a training function
# the input is the model, the number of epochs, optimzer, loss function, the dataloader, the sequence length, the batch size

def train(model, epochs, optimizer, criterion, dataloader, seq_len, batch_size):
    model.train()
    for epoch in range(epochs):
        h = model.init_hidden()
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            h = tuple([e.data for e in h])
            model.zero_grad()
            output, h = model(x, h)
            loss = criterion(output.view(batch_size * seq_len, -1), y.view(batch_size * seq_len))
            loss.backward()
            optimizer.step()
            
        print("Epoch: {}/{}...".format(epoch+1, epochs),
             "Loss: {:.4f}...".format(loss.item()))
    
    return model

In [54]:
# define a function to generate new text
# the input is the model, dataset, the size of the vocabulary, the number of characters to generate, the start string

def generate(model, dataset, vocab_size, size, start_str):
    model.eval()
    h = model.init_hidden()
    x = torch.tensor([dataset.vocab_to_int[c] for c in start_str]).to(DEVICE)
    x = x.view(1,-1)
    chars = [c for c in start_str]
    
    for i in range(size):
        h = tuple([e.data for e in h])
        output, h = model(x, h)
        # p = F.softmax(output, dim=1).data
        output = F.softmax(torch.squeeze(output), dim=0)
        dist = Categorical(output)
        index = dist.sample()
        # top_c = np.random.choice(vocab_size, 1, p=p.numpy().ravel())[0]
        chars.append(dataset.int_to_vocab[index.item()])
        x = torch.tensor([index.item()]).to(DEVICE)
        x = x.view(1,-1)
        
    return ''.join(chars)

In [13]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
SEQ_LEN = 10
N_EPOCHS = 100
N_HIDDEN = 512
N_LAYERS = 3
LEARNING_RATE = 0.001
EMBEDDING_DIM = 256

In [14]:
songdata = SongDataset(songs, SEQ_LEN)

In [15]:
trainloader = DataLoader(songdata, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [16]:
batch_iterator = iter(trainloader)
inputs, targets = next(batch_iterator)
print(inputs)
print(targets.shape)

tensor([[27,  3, 74, 66, 73, 76, 65, 86,  3,  0]])
torch.Size([1, 10])


In [17]:
model = RNN(N_HIDDEN, N_LAYERS, EMBEDDING_DIM, songdata.vocab_size, BATCH_SIZE, SEQ_LEN)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

In [18]:
model = model.to(DEVICE)

In [19]:
trained_model = train(model, N_EPOCHS, optimizer, loss_fn, trainloader, SEQ_LEN, BATCH_SIZE)

Epoch: 1/100... Loss: 2.9693...
Epoch: 2/100... Loss: 0.6176...
Epoch: 3/100... Loss: 0.7451...
Epoch: 4/100... Loss: 0.4989...
Epoch: 5/100... Loss: 0.3391...
Epoch: 6/100... Loss: 0.5361...
Epoch: 7/100... Loss: 0.4506...
Epoch: 8/100... Loss: 0.7060...
Epoch: 9/100... Loss: 0.4936...
Epoch: 10/100... Loss: 0.1470...
Epoch: 11/100... Loss: 0.3366...
Epoch: 12/100... Loss: 0.3389...
Epoch: 13/100... Loss: 0.0807...
Epoch: 14/100... Loss: 0.0545...
Epoch: 15/100... Loss: 0.2508...
Epoch: 16/100... Loss: 0.4100...
Epoch: 17/100... Loss: 0.4160...
Epoch: 18/100... Loss: 0.0096...
Epoch: 19/100... Loss: 0.0302...
Epoch: 20/100... Loss: 0.1840...
Epoch: 21/100... Loss: 0.3082...
Epoch: 22/100... Loss: 0.2188...
Epoch: 23/100... Loss: 0.4357...
Epoch: 24/100... Loss: 0.3181...
Epoch: 25/100... Loss: 0.5191...
Epoch: 26/100... Loss: 0.1370...
Epoch: 27/100... Loss: 0.2739...
Epoch: 28/100... Loss: 0.1727...
Epoch: 29/100... Loss: 0.6552...
Epoch: 30/100... Loss: 0.1421...
Epoch: 31/100... Lo

In [55]:
song = generate(trained_model, songdata, songdata.vocab_size, 500, "X")

In [56]:
print(song)

X: 0
T: Angelus Ad Virginum
Z: Public Domain
L: 1/4
M: 6/8
Q: 1/4=120
V: P1 name="melody"
V: P2 name="mlody"
V: P2 name="melody"
V: P2 name="melody"
V: P2 name="melody"
V: P2 name="melody"
V: 0
T: Angelus Ad Virginum
Z: Publus Ad Virginum
Z: Public Domain
L: 1/4
M: 6/8
Q: 1/4=120
V: P1 name="melody"
V: P2 name="me="melody"
V: P20
V: P1 dy"
V: P2 Public Domain
L: 1/4=120
V: P1 name="melody"
V: P2 name="melody"
V: P2 name="melody"
V: P2 name="melody"
V: P2 name="me="melody"
V: P2 name="melody"
V: P
